# Metrics Evaluator

### Introduction

To make sure that the splits don't make me choose a wrong model (since the difference of the finalist models is very little), I will run all of them many times to get a consistent metric from them, averaging its scores and stds.

In [1]:
# Load the necessary modules
# Relative paths (make sure you are in src)
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# plotting libraries
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

# Data management libraries 
import numpy as np 
import pandas as pd 
import math
import scipy.stats as stats

# My code
from data_processing import get_path
from Models.RandomForest import Random_Forest_ensemble
from Models.BaggingEnsemble import Bagging_Ensemble
from Models.LinearRegressor import LinearRegressor
from Models.LogisticRegressor import LogisticRegressor
from Models.LinearRegressionVariants import LR_Relations, LR_ensemble
from Models.SVR import SVR_manual
from utils import evaluate_classification_metrics, evaluate_regression_metrics, cross_validation, plot_residuals_color
from data_processing import standarize_numerical_variables

# Machine learning libraries 
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import BaggingClassifier, BaggingRegressor, RandomForestClassifier, RandomForestRegressor,StackingRegressor
from sklearn.metrics import accuracy_score, mean_absolute_error,mean_squared_error,confusion_matrix, make_scorer
from sklearn.tree import DecisionTreeRegressor, export_graphviz, DecisionTreeClassifier
from sklearn.decomposition import KernelPCA, PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression, RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import Isomap
from sklearn.datasets import make_regression
from sklearn.svm import SVR, SVC
from sklearn.utils import shuffle

# Parameters
import configparser
config = configparser.ConfigParser()
config.read('../configuration.ini')

['../configuration.ini']

In [2]:
# Import data from relative path for train
file = get_path('created_files','train',parent_dir)
data = pd.read_csv(file,sep = ',')
X_train_p = data.drop(columns=['T3']).reset_index(drop=True)
y_train_p = data['T3'].reset_index(drop=True)

# Do the train/ test split and standarize AFTER so there is no data leakage
X_train, X_test, y_train_unstd, y_test_unstd = train_test_split(X_train_p,y_train_p,test_size=0.2)
X_train, X_test, y_train,y_test,scaler_y = standarize_numerical_variables(X_train, X_test, y_train_unstd, y_test_unstd,1)


### Model 1: Final Models
Models to try out in the Metrics Evaluator

In [8]:
base_models = [
    #('lr', LinearRegression()),
    ('lasso', LassoCV(cv=5)),
    ('RF', RandomForestRegressor(random_state=42)),
    #('svm', SVR(C=1.0, epsilon=0.2))
]
stack = StackingRegressor(estimators=base_models, final_estimator=LinearRegression(), cv=5, passthrough=True)
models = [LinearRegression(),stack]
names = ['LinReg','Stack']



### Metrics Evaluator Code

In [9]:
final_scores = []

for model in models:
    scores = []
    stds = []
    scores_train = []
    MAE = []
    MSE = []
    for _ in range(300):
        X_shuffled, y_shuffled = shuffle(X_train_p, y_train_p, random_state=None)
        X_train, X_test, y_train_unstd, y_test_unstd = train_test_split(X_shuffled, y_shuffled, test_size=0.2)
        X_train, X_test, y_train, y_test, scaler_y = standarize_numerical_variables(X_train, X_test, y_train_unstd, y_test_unstd, 1)

        model.fit(X_train,y_train)
        predictions = model.predict(X_test)
        predictions_train = model.predict(X_train)

        score = evaluate_regression_metrics(y_test,y_train,predictions,predictions_train)
        scores.append(score['R² Score Test:'])
        scores_train.append(score['R² Score Train:'])
        MAE.append(score['Mean Absolute Error:'])
        MSE.append(score['Mean Squared Error:'])

    final_scores.append([np.mean(scores),np.std(scores),np.mean(MAE),np.mean(MSE),np.mean(scores_train)])

results_df = pd.DataFrame(final_scores, columns=['R² Mean Test', 'R² Std Test', 'Mean Absolute Error', 'Mean Squared Error', 'R² Mean Train'], index=names)


In [5]:
scores = []
scores_train = []
MAE = []
MSE = []
for _ in range(300):
    X_shuffled, y_shuffled = shuffle(X_train_p, y_train_p, random_state=None)
    X_train, X_test, y_train_unstd, y_test_unstd = train_test_split(X_shuffled, y_shuffled, test_size=0.2)
    X_train, X_test, y_train, y_test, scaler_y = standarize_numerical_variables(X_train, X_test, y_train_unstd, y_test_unstd, 1)

    LR_Rel = LR_Relations()
    LR_Rel.fit(X_train,X_test,y_train)
    predictions,score = LR_Rel.predict(y_test,y_train)
    scores.append(score['R² Score Test:'])
    scores_train.append(score['R² Score Train:'])
    MAE.append(score['Mean Absolute Error:'])
    MSE.append(score['Mean Squared Error:'])

final_score = [np.mean(scores),np.std(scores),np.mean(MAE),np.mean(MSE),np.mean(scores_train)]
results_df.loc['LR_Relations'] = final_score

In [6]:
scores = []
scores_train = []
MAE = []
MSE = []
for _ in range(300):
    X_shuffled, y_shuffled = shuffle(X_train_p, y_train_p, random_state=None)
    X_train, X_test, y_train_unstd, y_test_unstd = train_test_split(X_shuffled, y_shuffled, test_size=0.2)
    X_train, X_test, y_train, y_test, scaler_y = standarize_numerical_variables(X_train, X_test, y_train_unstd, y_test_unstd, 1)

    LR_E = LR_ensemble()
    LR_E.fit(X_train,X_test,y_train)
    predictions = LR_E.predict()
    score = LR_E.score_complete(y_test,y_train)
    scores.append(score['R² Score Test:'])
    scores_train.append(score['R² Score Train:'])
    MAE.append(score['Mean Absolute Error:'])
    MSE.append(score['Mean Squared Error:'])

final_score = [np.mean(scores),np.std(scores),np.mean(MAE),np.mean(MSE),np.mean(scores_train)]
results_df.loc['LR_Ensemble'] = final_score

In [7]:
print(results_df)

              R² Mean Test  R² Std Test  Mean Absolute Error  \
LinReg            0.830102     0.033044             0.251285   
LR_Relations      0.833837     0.031910             0.249648   
LR_Ensemble       0.833254     0.033965             0.249137   

              Mean Squared Error  R² Mean Train  
LinReg                  0.169390       0.853024  
LR_Relations            0.168190       0.854739  
LR_Ensemble             0.166946       0.852139  
